In [1]:
import sys
from pathlib import Path
import importlib.util
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import dataframe_image as dfi
import scipy.stats as stats
import numpy as np
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import spearmanr
from scipy.stats import mannwhitneyu
from pandas.plotting import table

# notebooks/.. = project_root
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


package_dir = project_root / "scdb_code_map"
init_file = package_dir / "__init__.py"
init_file.touch(exist_ok=True)  # creates empty __init__.py if missing


spec = importlib.util.find_spec("scdb_code_map")
if spec is None:
    raise ImportError(f"Cannot find 'scdb_code_map' in {project_root}")

In [2]:
conn = sqlite3.connect("../data/legal_sc.db")
c_issues = pd.read_sql("SELECT * FROM c_issues", conn)
c_votes = pd.read_sql("SELECT * FROM c_votes", conn)
c_dockets = pd.read_sql("SELECT * FROM c_dockets", conn)
c_citations = pd.read_sql("SELECT * FROM c_citations", conn)
conn.close()

In [10]:
# check size
print(c_issues.shape, c_votes.shape, c_dockets.shape, c_citations.shape)

(13930, 53) (14004, 53) (10857, 53) (9341, 53)


In [5]:
# take a look at tables' column names (all tables have the same columns, just print for one)
print(c_issues.columns.tolist())

['caseId', 'docketId', 'caseIssuesId', 'voteId', 'dateDecision', 'decisionType', 'usCite', 'sctCite', 'ledCite', 'lexisCite', 'term', 'naturalCourt', 'chief', 'docket', 'caseName', 'dateArgument', 'dateRearg', 'petitioner', 'petitionerState', 'respondent', 'respondentState', 'jurisdiction', 'adminAction', 'adminActionState', 'threeJudgeFdc', 'caseOrigin', 'caseOriginState', 'caseSource', 'caseSourceState', 'lcDisagreement', 'certReason', 'lcDisposition', 'lcDispositionDirection', 'declarationUncon', 'caseDisposition', 'caseDispositionUnusual', 'partyWinning', 'precedentAlteration', 'voteUnclear', 'issue', 'issueArea', 'decisionDirection', 'decisionDirectionDissent', 'authorityDecision1', 'authorityDecision2', 'lawType', 'lawSupp', 'lawMinor', 'majOpinWriter', 'majOpinAssigner', 'splitVote', 'majVotes', 'minVotes']


In [6]:
# for a given column, look at it's most common values

top_10 = c_citations['petitioner'].value_counts().head(10)

print(top_10)

petitioner
28.0     921
27.0     875
100.0    654
126.0    514
19.0     261
145.0    238
151.0    192
249.0    187
106.0    132
8.0      132
Name: count, dtype: int64


In [11]:
# for a given column, look at the values in the first 100 rows

first_100_values = c_citations['petitioner'].head(100).tolist()

print(first_100_values)

[198.0, 100.0, 209.0, 27.0, 27.0, 198.0, 148.0, 189.0, 4.0, 135.0, 126.0, 100.0, 27.0, 106.0, 338.0, 27.0, 102.0, 27.0, 369.0, 306.0, 306.0, 382.0, 142.0, 28.0, 27.0, 172.0, 177.0, 25.0, 27.0, 245.0, 354.0, 145.0, 126.0, 166.0, 145.0, 249.0, 241.0, 324.0, 171.0, 114.0, 27.0, 326.0, 195.0, 135.0, 172.0, 145.0, 27.0, 100.0, 25.0, 8.0, 28.0, 371.0, 100.0, 170.0, 123.0, 195.0, 382.0, 27.0, 231.0, 27.0, 123.0, 249.0, 19.0, 116.0, 171.0, 331.0, 189.0, 198.0, 237.0, 304.0, 371.0, 177.0, 366.0, 185.0, 253.0, 4.0, 100.0, 145.0, 245.0, 27.0, 27.0, 349.0, 158.0, 151.0, 240.0, 371.0, 135.0, 151.0, 122.0, 177.0, 389.0, 389.0, 184.0, 234.0, 100.0, 151.0, 369.0, 195.0, 195.0, 27.0]


In [7]:
# more raw table explorations

# Print first 10 rows and first 10 columns
#print("\nIssues preview (first 100 rows and 5 columns):")
#print(c_issues.iloc[:10, :5])

#print("\nDockets preview (first 10 rows and 5 columns):")
#print(c_dockets.iloc[:10, :5])

# For a unique case_ID, see how issue, vote, docket and citation table handles it differently
#case_ID_num = '1946-007'
case_ID_num = '1956-113'
issue_filtered = c_issues[c_issues['caseId'] == case_ID_num][['caseId', 'docket', 'issue', 'issueArea', 'lawType', 'lawSupp', 'splitVote']]
vote_filtered = c_votes[c_votes['caseId'] == case_ID_num][['caseId', 'docket', 'issue', 'issueArea', 'lawType', 'lawSupp', 'splitVote']]
docket_filtered = c_dockets[c_dockets['caseId'] == case_ID_num][['caseId', 'docket', 'issue', 'issueArea', 'lawType', 'lawSupp', 'splitVote']]
citation_filtered = c_citations[c_citations['caseId'] == case_ID_num][['caseId', 'docket', 'issue', 'issueArea', 'lawType', 'lawSupp', 'splitVote']]


print('\n', issue_filtered)
print('\n', vote_filtered)
print ('\n', docket_filtered)
print ('\n', citation_filtered)

# Intepretations
# issue table can dedicated multiple rows to a single unique case
# it shows different issue,  lawType (the main type of law), lawSupp (whether the case involves supplemental law), splitVote etc
# each of those unique combo produce a row
# issue table that include split vote will show the value 2 in splitVote column
# docket table only shows one row for each docket, so if a single case has two dockets, it has two rows
# citation table only show one row for each unique case


         caseId docket    issue  issueArea  lawType  lawSupp  splitVote
2041  1956-113      6  30060.0        3.0      3.0    369.0          1
2042  1956-113      7  30060.0        3.0      3.0    369.0          1
2043  1956-113      8  30060.0        3.0      3.0    369.0          1
2044  1956-113      8  30060.0        3.0      5.0    507.0          1

         caseId docket    issue  issueArea  lawType  lawSupp  splitVote
2045  1956-113      6  30060.0        3.0      3.0    369.0          1
2046  1956-113      6  30060.0        3.0      3.0    369.0          2
2047  1956-113      7  30060.0        3.0      3.0    369.0          1
2048  1956-113      7  30060.0        3.0      3.0    369.0          2
2049  1956-113      8  30060.0        3.0      3.0    369.0          1
2050  1956-113      8  30060.0        3.0      3.0    369.0          2
2051  1956-113      8  30060.0        3.0      5.0    507.0          1

         caseId docket    issue  issueArea  lawType  lawSupp  splitVote


In [8]:
# splitVote is relatively rare
# count how many rows actually have the value 2 in the case centered organized by issue including splitvote table
# only 74 rows
vote_issue = c_votes[c_votes['splitVote'] != 1][['caseId', 'splitVote']]
print(vote_issue)

# percentage of case with splitvote
percent_splitcase = len(vote_issue)/len(c_citations)
print(f"The percentage of cases that involve splitvote is: {percent_splitcase*100}%")

         caseId  splitVote
267    1946-134          2
329    1947-035          2
669    1948-091          2
1297   1952-034          2
2046   1956-113          2
...         ...        ...
12902  2011-077          2
13039  2013-038          2
13068  2013-062          2
13081  2013-068          2
13354  2016-070          2

[74 rows x 2 columns]
The percentage of cases that involve splitvote is: 0.7922064018841666%


In [9]:
# double check if I remove duplicate caseID (rows with the same caseID) from all tables, how would that change their shape

no_dup_issue = c_issues.drop_duplicates(subset=['caseId'])
no_dup_vote = c_votes.drop_duplicates(subset=['caseId'])
no_dup_docket = c_dockets.drop_duplicates(subset=['caseId'])
no_dup_citation = c_citations.drop_duplicates(subset=['caseId'])

print(c_issues.shape, c_votes.shape, c_dockets.shape, c_citations.shape)
print(no_dup_issue.shape, no_dup_vote.shape, no_dup_docket.shape, no_dup_citation.shape)

# Intepretation
# If removing duplicate caseID, all tables will have the same number of rows = original row number in citation table
# so there are 9341 unique cases in the database

(13930, 53) (14004, 53) (10857, 53) (9341, 53)
(9341, 53) (9341, 53) (9341, 53) (9341, 53)
